# Lesson 3: Building an Agent Reasoning Loop

## Setup

In [ ]:
from helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [ ]:
import nest_asyncio
nest_asyncio.apply()

## Load the data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

**Note**: The pdf file is included with this lesson. To access it, go to the `File` menu and select`Open...`.

## Setup the Query Tools

In [ ]:
from utils import get_doc_tools

vector_tool, summary_tool = get_doc_tools("metagpt.pdf", "metagpt")

## Setup Function Calling Agent

In [ ]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool],
    llm=llm,
    verbose=True
)
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
Agent roles in MetaGPT include the Product Manager, Architect, Project Manager, Engineer, and QA Engineer. The Product Manager is responsible for analyzing user requirements and creating the Product Requirement Document (PRD). The Architect translates these requirements into system design components. The Project Manager distributes tasks based on the system design, while Engineers execute the designated classes and functions. The QA Engineer formulates test cases to ensure code quality. These roles collaborate within a structured workflow to efficiently develop software solutions within the MetaGPT framework.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "communication between agent roles in MetaGP

In [ ]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: metagpt.pdf
file_path: metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2025-11-10
last_modified_date: 2024-06-24

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,
Liyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8
1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University,4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University,6University of Pennsylvania,
7University of California, Berkeley,8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to more
complex tasks, however, 

In [ ]:
response = agent.chat(
    "Tell me about the evaluation datasets used."
)

Added user message to memory: Tell me about the evaluation datasets used.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "evaluation datasets used in MetaGPT"}
=== Function Output ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, and the SoftwareDev dataset.
=== LLM Response ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, and the SoftwareDev dataset.


In [ ]:
response = agent.chat("Tell me the results over one of the above datasets.")

Added user message to memory: Tell me the results over one of the above datasets.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "results over HumanEval dataset"}
=== Function Output ===
MetaGPT achieved 85.9% and 87.7% Pass rates over the HumanEval dataset.
=== LLM Response ===
MetaGPT achieved 85.9% and 87.7% pass rates over the HumanEval dataset.


## Lower-Level: Debuggability and Control

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool],
    llm=llm,
    verbose=True
)
agent = AgentRunner(agent_worker)

In [ ]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

In [ ]:
step_output = agent.run_step(task.task_id)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
Agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. The Product Manager is responsible for analyzing user requirements and formulating a detailed Product Requirement Document (PRD). The Architect translates requirements into system design components. The Project Manager handles task distribution, while Engineers execute designated classes and functions based on the design. The QA Engineer formulates test cases to ensure code quality. These roles work together in a sequential workflow following Standard Operating Procedures (SOPs) to efficiently tackle complex tasks within the MetaGPT framework.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how age

In [ ]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task 382cc08c-5242-4f65-9c48-e0d9ace02771: 1
Agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. The Product Manager is responsible for analyzing user requirements and formulating a detailed Product Requirement Document (PRD). The Architect translates requirements into system design components. The Project Manager handles task distribution, while Engineers execute designated classes and functions based on the design. The QA Engineer formulates test cases to ensure code quality. These roles work together in a sequential workflow following Standard Operating Procedures (SOPs) to efficiently tackle complex tasks within the MetaGPT framework.


In [ ]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task 382cc08c-5242-4f65-9c48-e0d9ace02771: 1


TaskStep(task_id='382cc08c-5242-4f65-9c48-e0d9ace02771', step_id='16dd9e62-7df9-4d72-bb4f-bbfc84f363e0', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)

In [ ]:
step_output = agent.run_step(
    task.task_id, input="What about how agents share information?"
)

Added user message to memory: What about how agents share information?
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents share information in MetaGPT"}
=== Function Output ===
Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool where agents publish structured messages and subscribe to relevant messages based on their profiles. This protocol enhances role communication efficiency by allowing agents to obtain directional information from other roles and public information from the environment. Additionally, agents can use an effective publish-subscribe mechanism to exchange information and collaborate effectively within the framework. The shared message pool allows all agents to exchange information transparently without the need to inquire about other agents individually, and agents utilize a subscription mechanism based on their role-specific interests to extract relevant info

In [ ]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

=== LLM Response ===
Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool where agents publish structured messages and subscribe to relevant messages based on their profiles. This protocol enhances role communication efficiency by allowing agents to obtain directional information from other roles and public information from the environment. Additionally, agents can use an effective publish-subscribe mechanism to exchange information and collaborate effectively within the framework. The shared message pool allows all agents to exchange information transparently without the need to inquire about other agents individually, and agents utilize a subscription mechanism based on their role-specific interests to extract relevant information, ensuring they receive only task-related information and avoid distractions from irrelevant details.
True


In [ ]:
response = agent.finalize_response(task.task_id)

In [ ]:
print(str(response))

assistant: Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool where agents publish structured messages and subscribe to relevant messages based on their profiles. This protocol enhances role communication efficiency by allowing agents to obtain directional information from other roles and public information from the environment. Additionally, agents can use an effective publish-subscribe mechanism to exchange information and collaborate effectively within the framework. The shared message pool allows all agents to exchange information transparently without the need to inquire about other agents individually, and agents utilize a subscription mechanism based on their role-specific interests to extract relevant information, ensuring they receive only task-related information and avoid distractions from irrelevant details.
